In [ ]:
## get initial set of labels from karma3

In [6]:
import pandas as pd
from google.cloud import bigquery
import pandas as pd
import glob
client = bigquery.Client()

savepath = f"data"

In [3]:
import requests

response = requests.get("https://lens-api.k3l.io/profile_count?strategy=creator")
count = response.json()['count']

response = requests.get(f"https://lens-api.k3l.io/profile_scores?strategy=creator&limit={count}")
karma_data = response.json()
karma_data = pd.DataFrame(karma_data)
karma_data.to_csv('data/karma_data.csv')
del response
karma_data.head()

,score,rank,handle,followersCount,id
0,0.064284,1,stani.lens,57607,5
1,0.059850,2,bradorbradley.lens,33082,36
2,0.059520,3,christina.lens,28154,142
3,0.055198,4,blackdave.lens,819,36284
4,0.054165,5,chriscomrie.lens,4917,73557


In [4]:
karma_data.shape

(100004, 5)

In [5]:
## get bigquery data

In [13]:


def dispatcher_data_query():
    return """
SELECT
  profile_id,
  profile_dispatcher_address,
  block_timestamp AS dispatcher_setup_timestamp
FROM
  `lens-public-data.polygon.public_profile_dispatcher`
"""




# Perform a query.
QUERY = dispatcher_data_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/dispatcher_data')
df.shape

(82988, 3)

In [11]:

def aggregated_stats_query():
    return f"""
SELECT
  profile_id,
  total_posts,
  total_comments,
  total_mirrors,
  total_collects,
  datastream_metadata.source_timestamp
FROM
  `lens-public-data.polygon.public_profile_stats`
"""

# Perform a query.
QUERY = aggregated_stats_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/aggregated_stats')
df.shape

(116483, 6)

In [12]:
def ens_address_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.ens_address_reverse_record`
"""

# Perform a query.
QUERY = ens_address_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/ens_address')
df.shape

(47508, 1)

In [19]:
def proof_of_humanity_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.proof_of_humanity_address_to_proof_record`
WHERE
  is_registered=true
"""

# Perform a query.
QUERY = proof_of_humanity_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/proof_of_humanity')
df.shape

(855, 1)

In [20]:
def sybil_dot_org_address_verified_twitter_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.sybil_dot_org_address_verified_twitter`
"""

# Perform a query.
QUERY = sybil_dot_org_address_verified_twitter_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/sybil_dot_org_address_verified_twitter')
df.shape

(2784, 1)

In [22]:
def worldcoin_human_check_query():
    return f"""
SELECT
  profile_id
FROM
  `lens-public-data.polygon.worldcoin_human_check`
WHERE
  verified=true
"""

# Perform a query.
QUERY = worldcoin_human_check_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/worldcoin_human_check')
df.shape

(54, 1)

In [24]:
def most_used_app_query():
    return f"""
SELECT
  profile_id,
  ARRAY_AGG( STRUCT(app_id,
      total_posts)
  ORDER BY
    total_posts DESC
  LIMIT
    1 )[
OFFSET
  (0)].app_id AS app_id_highest_posts,
  ARRAY_AGG( STRUCT(app_id,
      total_posts)
  ORDER BY
    total_comments DESC
  LIMIT
    1 )[
OFFSET
  (0)].app_id AS app_id_highest_comments,
FROM
  `lens-public-data.polygon.public_app_profile_stats`
GROUP BY
  profile_id
"""

# Perform a query.
QUERY = most_used_app_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/most_used_app')
df.shape

(94732, 3)

In [4]:
import numpy as np
def q90(x):
    return x.quantile(0.9)
def q10(x):
    return x.quantile(0.1)
def q25(x):
    return x.quantile(0.25)
def q75(x):
    return x.quantile(0.75)


In [7]:
def post_data_insights_query():
    return f"""
SELECT
  profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_profile_post`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS TRUE
  AND is_related_to_post IS NULL
  AND is_related_to_comment IS null
"""

# Perform a query.
QUERY = post_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
df = df.sort_values(by=['profile_id','block_timestamp'],ascending=[True,False])
shifted = df.groupby("profile_id").shift(-1)
df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
df1 = df1.groupby('profile_id').agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})

df1.to_csv(f'{savepath}/post_data_insights')
df1.shape

running


(88645, 8)

In [8]:
def mirror_data_insights_query():
    return f"""
SELECT
  profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_profile_post`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS TRUE
  AND (is_related_to_post IS NOT NULL
    OR is_related_to_comment IS NOT NULL)
"""

# Perform a query.
QUERY = mirror_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
df = df.sort_values(by=['profile_id','block_timestamp'],ascending=[True,False])
shifted = df.groupby("profile_id").shift(-1)
df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
df1 = df1.groupby('profile_id').agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})

df1.to_csv(f'{savepath}/mirror_data_insights')
df1.shape

running


(74423, 8)

In [11]:
def comment_data_insights_query():
    return f"""
SELECT
  comment_by_profile_id AS profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_post_comment`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS true
"""

# Perform a query.
QUERY = comment_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
df = df.sort_values(by=['profile_id','block_timestamp'],ascending=[True,False])
shifted = df.groupby("profile_id").shift(-1)
df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
df1 = df1.groupby('profile_id').agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})

df1.to_csv(f'{savepath}/comment_data_insights')
df1.shape

running


(72763, 8)

In [12]:
def collect_data_insights_query():
    return f"""
SELECT
  collected_by AS address,
  block_timestamp
FROM
  `lens-public-data.polygon.public_publication_collect_module_collected_records`
"""

# Perform a query.
QUERY = collect_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
df = df.sort_values(by=['address','block_timestamp'],ascending=[True,False])
shifted = df.groupby("address").shift(-1)
df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
df1 = df1.groupby('address').agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})

df1.to_csv(f'{savepath}/collect_data_insights')
df1.shape

running


(226908, 8)

In [14]:
def paid_collect_data_insights_query():
    return f"""
WITH
  paid_pubs AS (
  SELECT
    publication_id
  FROM
    `lens-public-data.polygon.public_publication_collect_module_details`
  WHERE
    amount IS NOT NULL )
SELECT
  collected_by AS address,
  block_timestamp
FROM
  `lens-public-data.polygon.public_publication_collect_module_collected_records`
WHERE
  publication_id IN (
  SELECT
    publication_id
  FROM
    paid_pubs)
"""

# Perform a query.
QUERY = paid_collect_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
df = df.sort_values(by=['address','block_timestamp'],ascending=[True,False])
shifted = df.groupby("address").shift(-1)
df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
df1 = df1.groupby('address').agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})

df1.to_csv(f'{savepath}/paid_collect_data_insights')
df1.shape

running


(47881, 8)

In [97]:
def address_query():
    return f"""
SELECT
profile_id,
owned_by as address,
is_default,
metadata_block_timestamp,
block_timestamp
FROM
  `lens-public-data.polygon.public_profile`
"""

# Perform a query.
QUERY = address_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()

df.to_csv(f'{savepath}/address')
df.shape

(116487, 5)

# read data and make a csv

In [26]:
files = glob.glob(f'{savepath}/*')

In [22]:
for file in files[:1]:
    name = file.split('/')[-1]
    df = pd.read_csv(file,index_col=0)

In [27]:
files

['data/dispatcher_data',
 'data/post_data_insights',
 'data/collect_data_insights',
 'data/comment_data_insights',
 'data/ens_address',
 'data/mirror_data_insights',
 'data/most_used_app',
 'data/karma_data.csv',
 'data/sybil_dot_org_address_verified_twitter',
 'data/proof_of_humanity',
 'data/worldcoin_human_check',
 'data/aggregated_stats',
 'data/paid_collect_data_insights']

In [92]:
labels = pd.read_csv(f'{savepath}/karma_data.csv',index_col=0)


In [96]:
address

,profile_id,address,is_default,block_timestamp
0,0x01bc29,0x0784D6612b11943Dc5F6A609b69e6a49a4661Bed,False,NaN
1,0x01be2b,0xb27C9bF2cefE91325faCb311935816F82A6e2112,False,NaN
2,0x01be2c,0x2779A132721D6Ef4598f7C1fbcb19dB411E58124,False,NaN
3,0x01c4b8,0x22084410C6364eDD08FBbd9A9E3aE79f4016e92A,False,NaN
4,0x9435,0x21295CF3fD3665Cea1A79CdbeE5E4dd77DC287bF,False,2008.0
...,...,...,...,...
116482,0x4d69,0x886C48Fe5067Eb417a3D350d9862068184fC97dC,True,18855.0
116483,0x012a3e,0x87579E8F61e6b9e19C49de457EAB79644Fca399D,True,11852.0
116484,0x07aa,0x497dfDf509dC68CD05c509b9D68C34B6595916aD,True,19882.0
116485,0x53d1,0x0E324F7754C5B2A14AB924a81F70B1E2A3DeB654,True,18776.0


In [135]:
address = pd.read_csv(f'{savepath}/address',index_col=0)
address = address.drop({'block_timestamp'},axis=1)#.join( ((pd.to_datetime('2023-04-15',utc=True) - pd.to_datetime(address['block_timestamp'])).dt.total_seconds()/60/24).round(0))
address = address.drop({'metadata_block_timestamp'},axis=1)#.join( ((pd.to_datetime('2023-04-15',utc=True) - pd.to_datetime(address['metadata_block_timestamp'])).dt.total_seconds()/60/24).round(0))

address = address.rename({'block_timestamp':'days_since_profile_created', 'metadata_block_timestamp':'days_since_metadata_changed'},axis=1)



In [136]:
dispatcher = pd.read_csv(f'{savepath}/dispatcher_data',index_col=0)
dispatcher = dispatcher.drop(['profile_dispatcher_address','dispatcher_setup_timestamp'],axis=1).join(pd.get_dummies(dispatcher['profile_dispatcher_address'])).join( ((pd.to_datetime('2023-04-15',utc=True) - pd.to_datetime(dispatcher['dispatcher_setup_timestamp'])).dt.total_seconds()/60/24).round(0))
dispatcher = dispatcher.rename({'dispatcher_setup_timestamp':'days_since_dispatcher_changed'},axis=1)

profile_data = pd.merge(dispatcher,address, on='profile_id', how='outer')


worldcoin_human = pd.read_csv(f'{savepath}/worldcoin_human_check',index_col=0)
worldcoin_human['worldcoin_human']=True
profile_data = pd.merge(profile_data,worldcoin_human, on='profile_id', how='outer')

aggregated_stats = pd.read_csv(f'{savepath}/aggregated_stats',index_col=0)
profile_data = pd.merge(profile_data,aggregated_stats, on='profile_id', how='outer')



In [137]:
def read_insight_data(profile_data, name,group_col='profile_id'):
    profile_data.copy()
    df = pd.read_csv(f'{savepath}/{name}',index_col=0,skiprows=[0,2])
    df.columns = [f'{i}_{name}' for i in df.columns]
    profile_data = pd.merge(profile_data,df, left_on=group_col,right_index=True, how='outer')
    return profile_data

profile_data = read_insight_data(profile_data, name='post_data_insights')

profile_data = read_insight_data(profile_data, name='comment_data_insights')

profile_data = read_insight_data(profile_data, name='mirror_data_insights')



In [138]:
most_used_app = pd.read_csv(f'{savepath}/most_used_app',index_col=0)
profile_data = pd.merge(profile_data,most_used_app, on='profile_id', how='outer')


In [139]:
ens_address = pd.read_csv(f'{savepath}/ens_address',index_col=0)
ens_address['ens']=True

proof_of_humanity = pd.read_csv(f'{savepath}/proof_of_humanity',index_col=0)
proof_of_humanity['proof_of_humanity']=True

sybil_dot_org_address_verified_twitter = pd.read_csv(f'{savepath}/sybil_dot_org_address_verified_twitter',index_col=0)
sybil_dot_org_address_verified_twitter['sybil_dot_org_address_verified_twitter']=True

address_data = pd.merge(ens_address,proof_of_humanity, on='address',how='outer')
address_data = pd.merge(address_data,sybil_dot_org_address_verified_twitter, on='address',how='outer')

address_data = read_insight_data(address_data, name='collect_data_insights',group_col='address')

address_data = read_insight_data(address_data, name='paid_collect_data_insights',group_col='address')



In [171]:
data = profile_data.merge(address_data,on='address',how='outer').drop('address',axis=1)

In [172]:
data.shape

(265679, 70)

In [173]:
data = data.dropna(subset=['profile_id'])
data['id'] = data['profile_id'].apply(intify)
data = data.set_index('profile_id')
data.shape

(116487, 70)

In [174]:
data.head()

,0x112b57A293d99b79Fe360Af042bb3bfFc824Ab3a,0x25418Bcf53dAe1167cD5d0cad34Eec31C0C37E41,0x3530c7CAc2E47F27bA82a5d0D3671181171292DB,0x45F71328a415129CB25e3917e70f5BCA346C6151,0x46fAf146C4683663667af868dFc2F98ADc27fBd6,0x57B7bf6f792a6181Ec5aFB88cE7bcE330a9d1b67,0x62D0e168e35f87a1dB804C2b2daf63AE4D195cD1,0x6bC45934A408f7151093479bf59933cD3B75701d,0x761010EFc8826fFdcb8Ad005BD935698ed38DfE7,0x76D4fFdA8c66864FA07f0a285B8E01E11b4Bb7b3,...,min_collect_data_insights,mean_paid_collect_data_insights,median_paid_collect_data_insights,q10_paid_collect_data_insights,q25_paid_collect_data_insights,q75_paid_collect_data_insights,q90_paid_collect_data_insights,max_paid_collect_data_insights,min_paid_collect_data_insights,id
profile_id,,,,,,,,,,,,,,,,,,,,,
0xacae,False,False,False,True,False,False,False,False,False,False,...,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44206
0xf803,False,False,False,False,True,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63491
0xa738,False,False,False,False,False,False,False,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42808
0x8d06,False,False,False,True,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36102
0xe56e,False,False,False,True,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58734


In [175]:
def intify(x):
    return int(x,16)



In [176]:
data = data.merge(labels,on='id',how='left')

In [205]:
bad = data.dropna(subset=['rank','total_mirrors']).query('total_mirrors>100').sort_values(by='rank').tail(100).drop(labels.columns,axis=1)
good = data.sort_values(by='rank').head(100).drop(labels.columns,axis=1)
bad['label'] = 0
good['label'] = 1

training_data = pd.concat([bad,good])
validation_data = data.drop(training_data.index).drop(labels.columns,axis=1)

In [206]:
from sklearn.model_selection import train_test_split

In [207]:
X_train, X_test, y_train, y_test = train_test_split(training_data.drop('label',axis=1), training_data['label'], test_size=0.33, random_state=42)